# Weather & Calendar Data only
As previously seen in the data analysis, if we assume a loss function with mean squared error, we could actually improve the forecast by just adding a certain value.
As showed in the theoretical part, we will concetrating on minimizing the mean squared error and evaluation will be based both on the root mean squared (forecast) error (RMSE) and the MAPE (Mean Average Percentag Error).

In [1]:
# module imports
import os
import sys
import math
import itertools
import time as t
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import datetime
from numpy import newaxis


# Setup for Latex Export: https://matplotlib.org/users/pgf.html. Need to import before pyplot
def figsize(scale):
    fig_width_pt = 469.755                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width*golden_mean              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

import matplotlib as mpl
mpl.use('pgf')
pgf_with_rc_fonts = {
    "text.usetex": True,
    "font.family": "serif",
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(0.9),     # default fig size of 0.9 textwidth
    #"font.serif": [],                   # use latex default serif font
    #"font.sans-serif": ["DejaVu Sans"], # use a specific sans-serif font
}
mpl.rcParams.update(pgf_with_rc_fonts)

import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.tsa import stattools
from tabulate import tabulate

import math
import keras as keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
from keras.callbacks import TensorBoard
from keras.utils import np_utils
from keras.models import load_model

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
#from seasonal import fit_seasons, adjust_seasons


from IPython.display import HTML
from IPython.display import display
%matplotlib notebook
mpl.rcParams['figure.figsize'] = (9,5)

# Make plots nicer
#import seaborn
#%config InlineBackend.figure_format = 'retina'

# Import custom module functions
module_path = os.path.abspath(os.path.join(''))
if module_path not in sys.path:
    sys.path.append(module_path)

from helper_functions import data
from helper_functions import lstm

# Set up logging. Create path for log file
if not os.path.exists('logs/'):
    os.makedirs('logs')
logname = 'logs/notebook_05_' + t.strftime("%Y%m%d_%H%M") + '.py'
mode = 'rotate'
options = '-o -t'
%logstart $options $logname $mode

Using TensorFlow backend.


Activating auto-logging. Current session state plus future input saved.
Filename       : logs/notebook_05_20170507_2216.py
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


## Model run configuration


In [2]:
# Which features from the dataset should be loaded:
# ['all', 'actual', 'entsoe', 'weather_t', 'weather_i', 'holiday', 'weekday', 'hour', 'month']
features = ['weather_t', 'weather_i', 'holiday', 'weekday', 'hour', 'month']

# How the data should get splitted into training (+validation) and test
splits = [0.8]
validation_split = 0.2

# How many epochs in total
epochs = 50
# Set verbosity level. 0 for only per model, 1 for progress bar...
verbose = 0

# Output files
model_name = 'model5_'
res_dir = 'results/notebook_05/'
plot_dir = 'plots/notebook_05/'
model_dir = 'models/notebook_05/'
os.makedirs(res_dir, exist_ok=True)
os.makedirs(plot_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
results = pd.DataFrame(columns=['model_name', 'config', 'dropout',
                                'train_loss', 'train_rmse', 'train_mae', 'train_mape', 
                                'valid_loss', 'valid_rmse', 'valid_mae', 'valid_mape', 
                                'test_rmse', 'test_mae', 'test_mape',
                                'epochs', 'batch_train', 'input_shape',
                                'total_time', 'time_step', 'splits'
                               ])
output_table = res_dir + model_name + 'results_' + t.strftime("%Y%m%d") + '.csv'
test_output_table = res_dir + model_name + 'test_results' + t.strftime("%Y%m%d") + '.csv'

## Loading the data:


In [3]:
# Input data
df_X = data.load_data(typ='standardized', features=features)
# actual values
df_y = data.load_data(typ='standardized', features=['actual'])
# The benchmark, which will be used to compare the model's forecast on the testdata
df_bench = data.load_data(typ='standardized', features=['entsoe'])

split_X = data.split_series(series=df_X, mode='percentage', splits=splits)
split_y = data.split_series(series=df_y, mode='percentage', splits=splits)
split_bench = data.split_series(series=df_bench, mode='percentage', splits=splits)

time_train = split_y[0].index
time_test = split_y[1].index

X_train = split_X[0].values
X_test = split_X[1].values

y_train = split_y[0].values
y_test = split_y[1].values

benchmark_test = split_bench[1].values

| Loaded dataset  | standardized                                                      |
| File path       | /home/ubuntu/STLF/Data/fulldataset_stand.csv                      |
| Loaded features | ['weather_t', 'weather_i', 'holiday', 'weekday', 'hour', 'month'] |
| Dataset Shape   | (20231, 76)                                                       |
| Loaded dataset  | standardized                                 |
| File path       | /home/ubuntu/STLF/Data/fulldataset_stand.csv |
| Loaded features | ['actual']                                   |
| Dataset Shape   | (20231, 1)                                   |
| Loaded dataset  | standardized                                 |
| File path       | /home/ubuntu/STLF/Data/fulldataset_stand.csv |
| Loaded features | ['entsoe_fc']                                |
| Dataset Shape   | (20231, 1)                                   |
| Original dataset shape  | (20231, 76) |
| 1. new dataset of shape | (16184, 76) |
| 2. new dataset of shape | 

### Model 1: 1- 2 layers

In [ ]:
# Layers: Param: Stateful --> True, 
layer_conf = [ True, True, True ]

cells = [[ 10, 20, 24, 50, 100, 150 ], [0, 10, 20, 50, 100], [0, 10, 20, 50]]
dropout = [0, 0.1, 0.2]
batch_size = [8, 16, 64]
timesteps = [1]
# Based on these inputs, it will generate all possible combinations
#cells = [ [ 20, 30, 50 ], [ 0, 5, 10, 20 ] ]
#dropout = [ 0, 0.1, 0.2 ]
#batch_size = [ 1, 16, 32, 64 ]
#timesteps = [ 1 ]

models = []
models = lstm.generate_combinations(
    model_name=model_name, layer_conf=layer_conf, cells=cells, dropout=dropout, 
    batch_size=batch_size, timesteps=timesteps)



| Number of model configs generated | 1080 |


## Running through all generated models
Note: Depending on the above settings, this can take very long!

In [ ]:
start_time = t.time()
for idx, m in enumerate(models):
    stopper = t.time()
    print('========================= Model {}/{} ========================='.format(idx+1, len(models)))
    print(tabulate([['Starting with model', m['name']], ['Starting time', datetime.fromtimestamp(stopper)]],
                   tablefmt="jira", numalign="right", floatfmt=".3f"))
    try:
        # Creating the Keras Model
        model = lstm.create_model(layers=m['layers'], sample_size=X_train.shape[0], batch_size=m['batch_size'], 
                          timesteps=m['timesteps'], features=X_train.shape[1])
        # Training...
        history = lstm.train_model(model=model, mode='fit', y=y_train, X=X_train, 
                                   batch_size=m['batch_size'], timesteps=m['timesteps'], epochs=epochs, 
                                   rearrange=False, validation_split=validation_split, verbose=verbose)

        # Generating plots...
        lstm.plot_history(model_config=m, history=history, path=plot_dir, display=False)
        
        # Write results
        min_loss = np.min(history.history['val_loss'])
        min_idx = np.argmin(history.history['val_loss'])
        min_epoch = min_idx + 1
        
        print('______________________________________________________________________')
        print(tabulate([['Minimum validation loss at epoch', min_epoch, 'Time: {}'.format(t.time()-stopper)],
                        ['Training loss & MAE', history.history['loss'][min_idx], history.history['mean_absolute_error'][min_idx]  ], 
                        ['Validation loss & mae', history.history['val_loss'][min_idx], history.history['val_mean_absolute_error'][min_idx] ],
                       ], tablefmt="jira", numalign="right", floatfmt=".3f"))
        
        
        result = [{'model_name': m['name'], 'config': m, 'train_loss': history.history['loss'][min_idx], 'train_rmse': 0,
                   'train_mae': history.history['mean_absolute_error'][min_idx], 'train_mape': 0,
                   'valid_loss': history.history['val_loss'][min_idx], 'valid_rmse': 0, 
                   'valid_mae': history.history['val_mean_absolute_error'][min_idx],'valid_mape': 0, 
                   'test_rmse': 0, 'test_mae': 0, 'test_mape': 0, 'epochs': '{}/{}'.format(min_epoch, epochs), 'batch_train':m['batch_size'],
                   'input_shape':(X_train.shape[0], timesteps, X_train.shape[1]), 'total_time':t.time()-stopper, 
                   'time_step':0, 'splits':splits, 'dropout': m['layers'][0]['dropout']
                  }]
        results = results.append(result, ignore_index=True)
        
        # Saving the weights
        model.save(model_dir + m['name'] + '.h5')
        
        if not os.path.isfile(output_table):
            results.to_csv(output_table, sep=';')
        else: # else it exists so append without writing the header
            results.to_csv(output_table,mode = 'a',header=False, sep=';')
        
    # Shouldn't catch all errors, but for now...
    except BaseException as e:
        print('=============== ERROR {}/{} ============='.format(idx+1, len(models)))
        print(tabulate([['Model:', m['name']], ['Config:', m]], tablefmt="jira", numalign="right", floatfmt=".3f"))
        print('Error: {}'.format(e))
        result = [{'model_name': m['name'], 'config': m, 'train_loss': str(e)}]
        results = results.append(result, ignore_index=True)
        results.to_csv(output_table,sep=';')
        continue
        

========================= Model 1/1080 =========================
| Starting with model | model5_1_l-10              |
| Starting time       | 2017-05-07 22:16:06.212707 |
______________________________________________________________________
| Minimum validation loss at epoch | 2.000 | Time: 59.51445698738098 |
| Training loss & MAE              | 0.551 | 0.4850422566289218      |
| Validation loss & mae            | 0.175 | 0.330705162404496       |
========================= Model 2/1080 =========================
| Starting with model | model5_2_l-10              |
| Starting time       | 2017-05-07 22:17:05.816657 |
Warnining: Division "sample_size/batch_size" not a natural number.
Dropped the last 8 of 16184 number of obs.
Effective validation split now is: 0.200
______________________________________________________________________
| Minimum validation loss at epoch | 3.000 | Time: 29.661235094070435 |
| Training loss & MAE              | 0.522 | 0.45705939005088747      |
| Valida

========================= Model 16/1080 =========================
| Starting with model | model5_16_l-10_l-10_d-0.2  |
| Starting time       | 2017-05-07 22:25:26.105984 |
______________________________________________________________________
| Minimum validation loss at epoch | 3.000 | Time: 81.81735491752625 |
| Training loss & MAE              | 0.508 | 0.4561568994227019      |
| Validation loss & mae            | 0.138 | 0.29526728045793227     |
========================= Model 17/1080 =========================
| Starting with model | model5_17_l-10_l-10_d-0.2  |
| Starting time       | 2017-05-07 22:26:49.078684 |
Warnining: Division "sample_size/batch_size" not a natural number.
Dropped the last 8 of 16184 number of obs.
Effective validation split now is: 0.200
______________________________________________________________________
| Minimum validation loss at epoch | 3.000 | Time: 46.347596645355225 |
| Training loss & MAE              | 0.585 | 0.5061068529233208       |
| Vali

# Model selection based on MAE

# Mean Absolute Error:
http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error

In [ ]:
selection = 5
top_models = results.nsmallest(selection, 'valid_mae')

# Load the top models

In [ ]:
# Config
test_results = pd.DataFrame(columns=['Model name', 'Mean absolute error', 'Mean squared error', 'Diff. MAE', 'Diff. MSE'])

# Create ENTSOE benchmark for the forecast error
# TODO: For a fair comparison: We would need to cut the benchmark aswell. But as there are multiple models, we cant really...
mse_entsoe = mean_squared_error(y_test, benchmark_test)
mae_entsoe = mean_absolute_error(y_test, benchmark_test)

predictions = {}

for index, row in top_models.iterrows():
    filename = model_dir + row['model_name'] + '.h5'
    model = load_model(filename)
    batch_size = int(row['batch_train'])
    
    # Calculate scores
    loss, mae = lstm.evaluate_model(model=model, X=X_test, y=y_test, batch_size=batch_size, timesteps=1, verbose=verbose)
    
    # Store results
    result = [{'Model name': row['model_name'], 
               'Mean squared error': loss, 'Mean absolute error': mae,
               'Diff. MAE': mae - mae_entsoe, 'Diff. MSE': loss - mse_entsoe
              }]
    test_results = test_results.append(result, ignore_index=True)
    
    # Generate predictions
    model.reset_states()
    model_predictions = lstm.get_predictions(model=model, X=X_test, batch_size=batch_size, timesteps=timesteps[0], verbose=verbose)
    
    # Plot
    predictions[row['model_name']] = model_predictions
    

test_results = test_results.sort_values('Mean absolute error', ascending=True)
test_results = test_results.set_index(['Model name'])

if not os.path.isfile(test_output_table):
    test_results.to_csv(test_output_table, sep=';')
else: # else it exists so append without writing the header
    test_results.to_csv(test_output_table,mode = 'a',header=False, sep=';')

In [ ]:
print('Test dataset performance of the best {} (out of {} tested models):'.format(min(selection, len(models)), len(models)))
print('ENTSOE Forecast (Benchmark) metrics: \tMAE = {:.3f}  \tMSE = {:.3f}'.format(np.asscalar(mae_entsoe), np.asscalar(mse_entsoe)))
print(tabulate(test_results, headers='keys', tablefmt="grid", numalign="right", floatfmt=".3f"))

# Transform to original values (Destandardization)

$$z = \frac{x - \mu}{\sigma} \rightarrow x = z\sigma + \mu $$

In [ ]:
best_model = test_results.index[0]

# Load standardization params
columns = ['actual', 'entsoe_fc','bsl_t','brn_t','zrh_t','lug_t','lau_t','gen_t','stg_t','luz_t']
params_mu = read_csv(os.path.join('Data', 'standardization_params_mu.csv'), header=None)
params_mu.columns = columns
params_sigma = read_csv(os.path.join('Data', 'standardization_params_sigma.csv'), header=None)
params_sigma.columns = columns

mu = params_mu.loc[0]['actual']
sigma = params_sigma.loc[0]['actual']

y_test_raw = np.round(y_test * sigma + mu)
predictions_raw = np.round(predictions[best_model] * sigma + mu)
benchmark_test_raw = np.round(benchmark_test * sigma + mu)

size = predictions_raw.shape[0]
mse_entsoe = mean_squared_error(y_test_raw[0:size], benchmark_test_raw[0:size])
mae_entsoe = mean_absolute_error(y_test_raw[0:size], benchmark_test_raw[0:size])

mse = mean_squared_error(y_test_raw[0:size], predictions_raw)
mae = mean_absolute_error(y_test_raw[0:size], predictions_raw)

In [ ]:
time_vector = time_test.values
time_vector = time_vector[0:size]
time_vector = np.reshape(time_vector, (size,1))

plt.clf()
plt.ion()

#%matplotlib qt
plt.plot(time_vector, benchmark_test_raw[:size], label='ENTSOE Forecast')
plt.plot(time_vector, y_test_raw[:size], label='Actual Load')
plt.plot(time_vector, predictions_raw, label='Model predictions')
plt.title('LSTM Model using the ENTSOE Forecast as input')
plt.ylabel('Electricity load (in MW)')
plt.xlabel('Date')
plt.legend(loc='upper left')
plt.show

filename = plot_dir + model_name + 'top_model_predictions'
plt.savefig(filename + '.pgf')
plt.savefig(filename + '.pdf')


# Bad Comparison?
